In [1]:
import json
from datetime import datetime
from pathlib import Path

import pandas as pd
import requests
from sqlalchemy import select
from sqlalchemy.orm import aliased

from database import Database
from utils import DistributorApi

In [2]:
base_dir = Path(".").resolve()
data_dir = base_dir / "data"

In [3]:
api = DistributorApi()

In [4]:
rules = api.get_all_rules()
df_rules = pd.json_normalize(rules, sep="_")
df_rules = df_rules[
    [
        "id",
        "name",
        "description",
        "tag",
        "edit_state",
        "obsolete",
        "updated_by",
        "updated_on",
        "credential_level",
        "credential_list",
        "provider_level",
        "provider_list",
        "hotel_level",
        "hotel_list",
        "destination_level",
        "destination_list",
        "rate",
        "refundable",
        "market_level",
        "market_list",
        "meal_level",
        "meal_list",
        "check_in_level",
        "check_in_from",
        "check_in_to",
        "booking_date_level",
        "booking_date_from",
        "booking_date_to",
        "range_level",
        "range_from",
        "range_to",
        "max_release",
        "days_of_week_level",
        "days_of_week_list",
        "age",
        "room_level",
        "room_list",
        "num_of_nights_level",
        "num_of_nights_list",
        "hours_level",
        "hours_list",
        "dynamic_commission",
    ]
]

In [5]:
with pd.option_context("display.max_columns", None):
    display(df_rules.head())

,id,name,description,tag,edit_state,obsolete,updated_by,updated_on,credential_level,credential_list,provider_level,provider_list,hotel_level,hotel_list,destination_level,destination_list,rate,refundable,market_level,market_list,meal_level,meal_list,check_in_level,check_in_from,check_in_to,booking_date_level,booking_date_from,booking_date_to,range_level,range_from,range_to,max_release,days_of_week_level,days_of_week_list,age,room_level,room_list,num_of_nights_level,num_of_nights_list,hours_level,hours_list,dynamic_commission
0,686085,Blocked Provider for Hotel 6690,Automatic providers blocked algorithm.,0,1,False,AzureFunctions,12-10-2024 10:02:30,0,None,1,[CNN2],1,[6690],0,None,0,0,0,None,0,None,0,None,None,0,None,None,0,0,0,0,0,None,0,0,None,0,None,0,None,False
1,431443,BL- Apartamentos Masaru,CDH-19736,1,1,False,maria.ulloa@traveltino.com,08-04-2024 17:47:04,1,"[11986, 11992, 11993, 33284, 10206, 11771, 119...",1,[BCONG],1,[806],0,[],0,0,0,None,0,None,0,None,None,0,None,None,0,0,0,0,0,None,0,0,None,0,None,0,None,False
2,395508,SnapTravel UAE - PTRJ2,Close PRTJ2 - Supplier Errors - (Gisela 03.01.24),4,1,False,gisela.purra@smyrooms.com,03-01-2024 14:24:22,1,"[33914, 33915]",1,[PTRJ2],0,None,0,[],0,0,0,None,0,None,0,None,None,0,None,None,0,0,0,0,0,None,0,0,None,0,None,0,None,False
3,414798,Cierre Agoda + HBS TPS,CDH-18980,1,1,False,maria.ulloa@traveltino.com,25-03-2024 16:17:10,1,"[33238, 5226, 8550, 11655, 11945]",1,[YPL],1,[2368],0,[],0,0,0,None,0,None,0,None,None,0,None,None,0,0,0,0,0,None,0,0,None,0,None,0,None,False
4,681035,Blocked Provider for Hotel 113878,Automatic providers blocked algorithm.,0,1,False,AzureFunctions,01-10-2024 10:12:32,0,None,1,"[DIDAB2C, TBOH, SUT, W2M2]",1,[113878],0,None,0,0,0,None,0,None,0,None,None,0,None,None,0,0,0,0,0,None,0,0,None,0,None,0,None,False


In [11]:
# Set filter to level 1 rules
filter_lvl1 = (
    (df_rules["tag"] == 1)
    & (df_rules["credential_level"] == 1)
    & (df_rules["provider_level"] == 1)
    & (df_rules["hotel_level"] == 1)
    & (df_rules["destination_level"] == 0)
    & (df_rules["rate"] == 0)
    & (df_rules["refundable"] == 0)
    & (df_rules["market_level"] == 0)
    & (df_rules["meal_level"] == 0)
    & (df_rules["check_in_level"] == 0)
    & (df_rules["booking_date_level"] == 0)
    & (df_rules["range_level"] == 0)
    & (df_rules["max_release"] == 0)
    & (df_rules["days_of_week_level"] == 0)
    & (df_rules["age"] == 0)
    & (df_rules["room_level"] == 0)
    & (df_rules["num_of_nights_level"] == 0)
    & (df_rules["hours_level"] == 0)
)

In [13]:
df_rules_lvl1 = df_rules[filter_lvl1].copy()
df_rules_lvl1.shape

(512, 42)

In [8]:
df_rules_lvl1["credential_count"] = df_rules_lvl1["credential_list"].apply(len)
df_rules_lvl1.insert(8, "credential_count", df_rules_lvl1.pop("credential_count"))

In [9]:
df_rules_lvl1["credential_list_tuple"] = df_rules_lvl1["credential_list"].apply(tuple)
df_rules_lvl1["duplicates"] = df_rules_lvl1.groupby("credential_list_tuple")[
    "credential_list_tuple"
].transform("count")

In [10]:
df_rules_lvl1.to_csv("df_rules_lvl1.csv", index=False)